# Una solución empresarial completa

## Ahora llevaremos nuestro proyecto del día 1 al siguiente nivel

### DESAFÍO EMPRESARIAL:

Crear un producto que genere un folleto para una empresa que se utilizará para posibles clientes, inversores y posibles reclutas.

Se nos proporcionará un nombre de empresa y su sitio web principal.

Consulte el final de este cuaderno para ver ejemplos de aplicaciones empresariales del mundo real.

Y recuerde: ¡siempre estoy disponible si tiene problemas o ideas! No dude en comunicarse conmigo.

In [1]:
# imports
# Si esto falla, verifica que esté ejecutándose desde un entorno "activado" con (llms) en el símbolo del sistema

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Inicialización y constantes and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!


In [8]:
# La clase para representar una Página Web

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, verify=False)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = {link.get('href') for link in soup.find_all('a')}
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [9]:
frog = Website("https://cursos.frogamesformacion.com")
print(frog.get_contents())
frog.links

d:\Anaconda\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cursos.frogamesformacion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Título de la Web:
Frogames
Contenido de la Web:
Ir al contenido principal
Frogames
Menú alternativo
Menú
Iniciar sesión
Ganadora del premio 'Enseñanza online de datos y competencias digitales más innovadora de Europa, 2023'
Pasión por
aprender
con los
mejores
En Frogames Formación te ayudamos a convertirte en todo un experto en: Programación de Videojuegos, Inteligencia Artificial, Machine Learning, Desarrollo de Apps, Data Science y mucho más.
Aprende mientras te diviertes
Cursos, Rutas y Suscripciones
Certificados de finalización
Qué encontrarás
dentro
de Frogames
Cursos online y formación de calidad para toda la família
Rutas temáticas
Rutas organizadas para que aprendas paso a paso, subiendo cada escalón e incrementando tus conocimientos adquiridos
Instructores Expertos
Con un equipo de profesionales y expertos en la materia que te acompañará a lo largo de todo el aprendizaje en la plataforma
Certificados blockchain
Títulos verificados por blockchain para cada habilidad que aprenda

['/pages/aviso-legal',
 '/bundles/ruta-videojuegos',
 '/bundles/ruta-inteligencia-artificial',
 'https://cursos.frogamesformacion.com/pages/privacy',
 '/collections/solidity',
 '/collections/habilidades-blandas',
 '/bundles/ruta-blockchain',
 '/courses/c-sharp-unity-introduccion',
 '/courses/machine-learning-az',
 '/collections/reporting',
 '/bundles/ruta-bases-datos',
 '/bundles/ruta-programacion',
 '/courses/15-dias-power-bi',
 '/bundles/ruta-analisis-datos',
 '/bundles/ruta-desarrollo-multiplataforma',
 '/courses/principios-solid',
 '/pages/nuestros-clientes',
 'https://cursos.frogamesformacion.com/collections',
 '/bundles/ruta-trading',
 '/courses/chatgpt',
 '/collections/unreal-engine',
 '/courses/trigonometria-desde-cero',
 '/courses/matematicas-ml',
 '/pages/politica-de-cookies',
 '/collections/tableau',
 'https://www.ciobulletin.com/magazine/frogames-formacion-quality-spanish-language-courses',
 'https://cursos.frogamesformacion.com/pages/terms',
 '/courses/shaders-unreal-engin

## Primer paso: hacer que GPT-4o-mini determine qué enlaces son relevantes

### Usar una llamada a gpt-4o-mini para leer los enlaces en una página web y responder en JSON estructurado.
Debería decidir qué enlaces son relevantes y reemplazar los enlaces relativos como "/about" con "https://company.com/about".
Usaremos "one shot prompting" en las que proporcionamos un ejemplo de cómo debería responder en la solicitud.

Este es un excelente caso de uso para un LLM, porque requiere una comprensión matizada. Imagínate intentar programar esto sin LLMs analizando la página web: ¡sería muy difícil!

Nota al margen: existe una técnica más avanzada llamada "Salidas estructuradas" en la que requerimos que el modelo responda de acuerdo con una especificación. Cubrimos esta técnica en la Semana 8 durante nuestro proyecto autónomo de inteligencia artificial Agentic.

In [10]:
link_system_prompt = "Se te proporciona una lista de enlaces que se encuentran en una página web. \
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, \
como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.\n"
link_system_prompt += "Debes responder en JSON como en este ejemplo:"
link_system_prompt += """
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}
"""

In [15]:
print(link_system_prompt)

Se te proporciona una lista de enlaces que se encuentran en una página web. Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.
Debes responder en JSON como en este ejemplo:
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}



In [45]:
def get_links_user_prompt(website):
    user_prompt = "Asume el rol de experto datos relevantes en documentos y web con más de 10 años de experiencia."
    user_prompt += f"Te pasaré una lista de enlaces de la página web '{website.url}'."
    user_prompt += "Por favor, decide cuáles de esos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. \
No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.\n"
    user_prompt += "Links (puede que algunos sean links relativos, recuerda, tiene que estar la url completa):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [12]:
print(get_links_user_prompt(frog))

Aquí hay una lista de enlaces de la página web https://cursos.frogamesformacion.com - Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.
Links (puede que algunos sean links relativos):
/pages/aviso-legal
/bundles/ruta-videojuegos
/bundles/ruta-inteligencia-artificial
https://cursos.frogamesformacion.com/pages/privacy
/collections/solidity
/collections/habilidades-blandas
/bundles/ruta-blockchain
/courses/c-sharp-unity-introduccion
/courses/machine-learning-az
/collections/reporting
/bundles/ruta-bases-datos
/bundles/ruta-programacion
/courses/15-dias-power-bi
/bundles/ruta-analisis-datos
/bundles/ruta-desarrollo-multiplataforma
/courses/principios-solid
/pages/nuestros-clientes
https://cursos.frogamesformacion.com/collections
/bundles/ruta-trading
/courses/chatgpt
/collections/unreal-engine
/courses/trigonometri

In [46]:
def get_links(url, model=None):
    website = Website(url)
    model_to_use = model or MODEL
    response = openai.chat.completions.create(
        model=model_to_use,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [47]:
anthropic = Website("https://anthropic.com")
anthropic.links

d:\Anaconda\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'anthropic.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\Anaconda\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.anthropic.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


['/team',
 '/news/claude-for-enterprise',
 'https://www.linkedin.com/company/anthropicresearch',
 '/claude/haiku',
 'https://support.anthropic.com/',
 '/legal/commercial-terms',
 '/news',
 'https://claude.ai/',
 '/legal/consumer-terms',
 '/jobs',
 'https://trust.anthropic.com/',
 '/careers',
 'https://twitter.com/AnthropicAI',
 '/enterprise',
 '/research',
 '/legal/privacy',
 'https://status.anthropic.com/',
 'https://www.anthropic.com/research#entry:8@1:url',
 '/api',
 'https://www.anthropic.com/claude',
 '/legal/aup',
 'mailto:press@anthropic.com',
 '/pricing',
 '/supported-countries',
 '/news/core-views-on-ai-safety',
 '/company',
 '/',
 '/news/3-5-models-and-computer-use',
 'https://www.youtube.com/@anthropic-ai',
 '/customers',
 '/claude',
 '/responsible-disclosure-policy',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/claude/sonnet']

In [48]:
get_links("https://anthropic.com")

d:\Anaconda\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'anthropic.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\Anaconda\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.anthropic.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'links': [{'type': 'Pagina Acerca de',
   'url': 'https://www.anthropic.com/company'},
  {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'Pagina de Empleos', 'url': 'https://www.anthropic.com/jobs'},
  {'type': 'Pagina de Investigación',
   'url': 'https://www.anthropic.com/research'},
  {'type': 'Pagina de Claude', 'url': 'https://www.anthropic.com/claude'},
  {'type': 'Pagina de Noticias', 'url': 'https://www.anthropic.com/news'}]}

## Segundo paso: ¡crea el folleto!

Reúne todos los detalles en otro mensaje para GPT4-o

In [16]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Links encontrados:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://anthropic.com"))

In [17]:
system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa\
y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
Incluye detalles sobre la cultura de la empresa, los clientes, las carreras/empleos y los cursos/packs para futuros empleos si tienes la información."

# O descomenta las líneas a continuación para obtener un folleto más humorístico: esto demuestra lo fácil que es incorporar el "tono":

# system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa \
# y crea un folleto breve, divertido y gracioso sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
#Incluye detalles sobre la cultura de la empresa, los clientes y los cursos/packs para futuros empleos si tienes la información."


In [20]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Estás mirando una empresa llamada: {company_name}\n"
    user_prompt += f"Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncar si tiene más de 20.000 caracteres
    return user_prompt

In [23]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Página de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Página de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Página de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Página de API', 'url': 'https://www.anthropic.com/api'}, {'type': 'Página de Precios', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'Página de Clientes', 'url': 'https://www.anthropic.com/customers'}]}


"Estás mirando una empresa llamada: Anthropic\nAquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\nLanding page:\nTítulo de la Web:\nHome \\ Anthropic\nContenido de la Web:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nStart using Claude to drive efficiency and create new revenue streams.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 2024\nAlignment\n·\nResearch\nConstitutional AI: Harmlessness from AI Feedback\nDec 15, 2022\nAnnouncements\nCore Views on AI Safety: When, Why, What, and H

In [33]:
from markdown_pdf import MarkdownPdf, Section
# import markdown2
# import pdfkit

def markdown_to_pdf(texto_markdown, pdf_output):
    # Crea una instancia de MarkdownPdf
    pdf = MarkdownPdf()

    # Crea una sección con el contenido de texto_markdown
    seccion = Section(texto_markdown)

    # Añade la sección al PDF
    pdf.add_section(seccion)

    # Guarda el PDF
    pdf.save(pdf_output)

In [34]:
markdown_to_pdf("# Este es el titulo ## Subtitulo Este es el texto", "tmp.pdf")

In [41]:
def create_brochure(company_name, url,model=None, pdffilename=None):
    model_to_use=model or MODEL
    print(model_to_use)
    response = openai.chat.completions.create(
        model=model_to_use,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    if pdffilename:
        markdown_to_pdf(result, f"{company_name}.pdf")

In [42]:
create_brochure("Anthropic", "https://anthropic.com")

gpt-4o-mini


In [37]:
create_brochure("Serviloc", "https://serviloc.es", True)

d:\Anaconda\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'serviloc.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\Anaconda\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'serviloc.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Links encontrados: {'links': [{'type': 'Página de contacto', 'url': 'https://serviloc.es/contacto/'}, {'type': 'Página de la empresa', 'url': 'https://serviloc.es/'}, {'type': 'Página de Instalaciones', 'url': 'https://serviloc.es/instalaciones/'}, {'type': 'Página de gestión de proyectos e instalación', 'url': 'https://serviloc.es/gestion-proyectos-instalacion/'}, {'type': 'Página sobre seguridad al volante', 'url': 'https://serviloc.es/la-seguridad-al-volante-en-los-proximos-anos/'}, {'type': 'Página de sistemas antirrobos', 'url': 'https://serviloc.es/sistemas-antirrobos-coches/'}, {'type': 'Página en LinkedIn', 'url': 'https://www.linkedin.com/company/serviloc-control-y-gesti%C3%B3n-de-flotas-sl/'}, {'type': 'Página de Instagram', 'url': 'https://www.instagram.com/servilocgps/'}]}


d:\Anaconda\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'serviloc.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\Anaconda\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'serviloc.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\Anaconda\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'serviloc.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\Anaconda\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: Insecur

# Folleto de Serviloc

## Sobre Serviloc
Serviloc es una empresa líder en la **gestión de flotas y sistemas telemáticos en vehículos**. Con más de **25 años de experiencia** en el sector, nos especializamos en la instalación de sistemas telemáticos y electrónicos para todo tipo de vehículos, garantizando soluciones a medida que optimizan la eficiencia y el control de tu flota. Nuestro compromiso con la innovación y la excelencia nos distingue como referentes en España.

---

## Nuestros Servicios
- **Instalación de Sistemas Telemáticos**: Realizamos instalaciones a domicilio de manera rápida y eficiente, adaptándonos a tus necesidades específicas.
- **Gestión de Proyectos**: Coordinación precisa desde el primer contacto hasta la ejecución, asegurando resultados exitosos.
- **Soporte Continuo**: Ofrecemos asistencia técnica después de la instalación, garantizando el funcionamiento óptimo de los sistemas.

---

## Cultura de la Empresa
En Serviloc, cultivamos una cultura basada en la **innovación**, **experiencia**, y **atención personalizada**. Nuestro equipo de expertos está comprometido con la calidad y la satisfacción del cliente, ofreciendo atención constante y soluciones personalizadas. Creemos en la formación continua y el desarrollo profesional de nuestros colaboradores.

---

## Clientes
Contamos con una diversa gama de clientes satisfechos, desde pequeñas empresas hasta grandes corporaciones, a quienes ayudamos a optimizar sus operaciones a través de tecnología de vanguardia. Nuestra atención personalizada garantiza que cada cliente reciba la mejor solución para sus necesidades.

---

## Oportunidades de Empleo
Serviloc está en constante crecimiento y busca individuos apasionados que deseen formar parte de nuestro equipo. Ofrecemos:
- **Carreras en Instalación y Telemática**: Posiciones enfocadas en la instalación de sistemas telemáticos, sistemas embarcados y electrónica.
- **Capacitación Continua**: Cursos y capacitación en tecnología IoT y gestión de flotas para impulsar tu carrera.

---

## Contáctanos
¿Tienes preguntas o te gustaría solicitar más información sobre nuestros servicios? No dudes en contactarnos:

- **Teléfono**: 935 705 176
- **Email**: info@serviloc.es
- **Dirección**: Carrer de Can Pantiquet, 69, 08100 Mollet del Vallès, Barcelona
- **Horarios**: Lunes a viernes: 8:00–17:00

---

## ¡Solicita tu Demo!
Empieza a optimizar la gestión de tu flota con una demostración personalizada. **[Solicita una demo aquí](#)**.

---

Serviloc: **Soluciones telemáticas a tu medida.**

## Por último, una pequeña mejora

Con un pequeño ajuste, podemos cambiar esto para que los resultados se transmitan desde OpenAI,
con la animación de máquina de escribir habitual


In [32]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("Anthropic", "https://anthropic.com")

## Aplicaciones empresariales

En este ejercicio, ampliamos el código del día 1 para realizar múltiples llamadas a LLM y generar un documento.

En términos de técnicas, este es quizás el primer ejemplo de patrones de diseño de Agentic AI, ya que combinamos múltiples llamadas a LLM. Esto se abordará más en la semana 2 y luego volveremos a Agentic AI de manera importante en la semana 8, cuando construyamos una solución Agent completamente autónoma.

En términos de aplicaciones empresariales, generar contenido de esta manera es uno de los casos de uso más comunes. Al igual que con el resumen, esto se puede aplicar a cualquier vertical empresarial. Escriba contenido de marketing, genere un tutorial de producto a partir de una especificación, cree contenido de correo electrónico personalizado y mucho más. Explore cómo puede aplicar la generación de contenido a su negocio e intente crear un prototipo de prueba de concepto.